In [5]:

from numpy import argmax
from pickle import load
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)
 
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions
 
# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features
 
# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)
 
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text
 
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    # step over the whole set
    for key, desc_list in descriptions.items():
        # generate description
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        # store actual and predicted
        references = [d.split() for d in desc_list]
        actual.append(references)
        print(yhat.split())
        predicted.append(yhat.split())
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
 
# prepare tokenizer on train set
 
# load training dataset (6K)
filename = 'Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)
 
# prepare test set
 
# load test set
filename = 'Flickr8k_text/Flickr_8k.testImages.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('features.pkl', test)
print('Photos: test=%d' % len(test_features))
 
# load the model
filename = 'model_19.h5'
model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 6000
Descriptions: train=6000
Vocabulary Size: 7579
Description Length: 34
Dataset: 1000
Descriptions: test=1000
Photos: test=1000


2021-11-18 15:31:26.907297: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-11-18 15:31:26.960853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 15:31:26.961489: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6575GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2021-11-18 15:31:26.961527: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-11-18 15:31:26.963624: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-11-18 15:31:26.965250: I tensorflow/stream_executo

['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'water', 'endseq']
['startseq', 'boy', 'in', 'pink', 'shirt', 'is', 'jumping', 'into', 'the', 'water', 'endseq']
['startseq', 'young', 'boy', 'in', 'red', 'shirt', 'is', 'jumping', 'into', 'the', 'air', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', 'in', 'the', 'snow', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'water', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', 'with', 'red', 'ball', 'in', 'the', 'grass', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'little', 'boy', 'in', 'red', 'shirt', 'is', 'playing', 'with', 'toy', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', 'in', 'the', 'water', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'is', 'climbing', 'rock', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shir

['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'water', 'endseq']
['startseq', 'two', 'children', 'are', 'playing', 'in', 'the', 'water', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', 'in', 'the', 'snow', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', 'in', 'the', 'water', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'sitting', 'on', 'bench', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', 'in', 'the', 'snow', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'man', 'in', 'black', 'is', 'riding', 'wave', 'endseq']
['startseq', 't

['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'on', 'the', 'street', 'endseq']
['startseq', 'two', 'children', 'are', 'playing', 'in', 'the', 'water', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq']
['startseq', 'two', 'boys', 'are', 'playing', 'in', 'the', 'field', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'is', 'rock', 'climbing', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'and', 'white', 'hat', 'is', 'standing', 'on', 'the', 'street', 'endseq']
['startseq', 'boy', 'in', 'blue', 'shirt', 'is', 'jumping', 'into', 'the', 'water', 'endseq']
['startseq', 'two', 'boys', 'are', 'playing', 'soccer', 'on', 'the', 'field', 'endseq']
['startseq', 'little', 'boy', 'in', 'red', 'shirt', 'is', 'playing', 'with', 'red', 'toy', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', 'in', 'the', 'water', 'endseq']
['startseq', 'two', 'do

...

['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'two', 'children', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'hat', 'and', 'black', 'and', 'white', 'hat', 'and', 'black', 'and', 'white', 'and', 'white', 'flag', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'on', 'the', 'street', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'woman', 'endseq']
['startseq', 'little', 'boy', 'in', 'red', 'shirt', 'is', 'playing', 'in', 'pool', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', 'in', 'the', 'snow', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq'

['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'woman', 'endseq']
['startseq', 'young', 'boy', 'in', 'red', 'shirt', 'is', 'jumping', 'into', 'the', 'air', 'endseq']
['startseq', 'two', 'boys', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'is', 'standing', 'on', 'the', 'beach', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'the', 'man', 'is', 'wearing', 'red', 'shirt', 'and', 'black', 'shirt', 'and', 'black', 'shirt', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'young', 'boy', 'in', 'red', 'shirt', 'is', 'jumping', 'into', 'the', 'air', 'endseq']
['startseq', 'man', 'in', 'red', 'jacket', 'is', 'climbing', 'up', 'rock', 'endseq']
['startseq', 'boy', 'in', 'blue', 'bathing', 'suit', 'is', 'swimming', 'in', 'pool', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', '

['startseq', 'man', 'in', 'red', 'shirt', 'is', 'riding', 'large', 'wave', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'walking', 'down', 'the', 'street', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'hat', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'is', 'riding', 'bike', 'on', 'dirt', 'path', 'endseq']
['startseq', 'man', 'in', 'red', 'gear', 'is', 'tricks', 'on', 'bike', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'on', 'the', 'street', 'endseq']
['startseq', 'boy', 'in', 'pink', 'shirt', 'is', 'jumping', 'into', 'the', 'water', 'endseq']
['startseq', 'boy', 'in', 'blue', 'shirt', 'is', 'jumping', 'into', 'pool', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq']
['startseq', 'two', 'dogs', 'are', 'playing', 'in', 'the', 'snow'

['startseq', 'man', 'in', 'red', 'shirt', 'is', 'standing', 'on', 'rock', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'woman', 'endseq']
['startseq', 'dog', 'is', 'running', 'through', 'the', 'snow', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'young', 'boy', 'in', 'red', 'shirt', 'is', 'jumping', 'into', 'the', 'air', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'boy', 'in', 'red', 'shirt', 'is', 'jumping', 'into', 'the', 'air', 'endseq']
['startseq', 'man', 'in', 'red', '

['startseq', 'two', 'dogs', 'are', 'playing', 'in', 'the', 'water', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'two', 'children', 'are', 'playing', 'in', 'pool', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', 'in', 'front', 'of', 'building', 'endseq']
['startseq', 'young', 'boy', 'in', 'red', 'shirt', 'is', 'jumping', 'into', 'the', 'air', 'endseq']
['startseq', 'dog', 'is', 'running', 'through', 'the', 'grass', 'endseq']
['startseq', 'boy', 'in', 'blue', 'bathing', 'suit', 'is', 'jumping', 'into', 'pool', 'endseq']
['startseq', 'two', 'girls', 'are', 'playing', 'in', 'the', 'grass', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'is', 'climbing', 'rock', 'endseq']
['startseq', 'man', 'in', 'red', 'shirt', 'and', 'black', 'shirt', 'is', 'standing', '